In [1]:
import pandas as pd
import numpy as np
import pickle
import urllib.request
import dateutil.parser
import dateutil.rrule
import datetime
from IPython.display import display, HTML

## Summary

These tables compare the footfall data at two strategic locations in Newcastle, one counter spanning the full width of Northumberland St, and a second on the corner of Blackett St and John Dobson St used as a thoroughfare to the shopping district.

For each day and each direction, the following percentages are given with respect to the average pedestrians per hour:
 * Change since the day before
 * Change since the same weekday last week
 * Change compared to the median for the same weekday calculated over the last year
 
Statistics are provided for the morning peak, afternoon peak, inter-peak period (essentially daytime outside of peak hours), and night. As footfall is unlikely to be evenly distributed across these periods, the statistics for the current period will be volatile while data is still coming in.

In [2]:
# Used across most of the plots for people flows
dateToday = datetime.datetime.combine(datetime.date.today(), datetime.datetime.min.time())
peopleCountInterval = 900
peopleCountFrames = pickle.load(open('../cache/recent-pedestrian-flows-pd.pkl', 'rb'))

# TODO: Make this reflect the last entry in the frame, not the time now
print('Last data obtained %s' 
    % (np.max(list(map(lambda f: np.max(f.index), peopleCountFrames.values()))).strftime('%d %B %Y %H:%M')))

Last data obtained 19 March 2020 09:15


In [3]:
# Ignore non-numeric columns in the dataframe
plottableTypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
perMinuteFactor = (peopleCountInterval / 60)

In [4]:
sensorsToSummarise = {
    'Northumberland St near TK Maxx': 'Newcastle: City centre shopping district (Northumberland St near TK Maxx)',
    'Pavement (south side) corner John Dobson St and Blackett St': 'Newcastle: City centre edge of shopping district (corner of Blackett St and John Dobson St)'
}

def classifyTime(t):
    hour = int(t.strftime('%H'))
    if hour < 7:
        return 'Night (19:00 - 07:00)'
    elif hour < 10:
        return 'Morning peak (07:00 - 10:00)'
    elif hour < 16:
        return 'Inter-peak (10:00 - 16:00)'
    elif hour < 19:
        return 'Evening peak (16:00 - 19:00)'
    else:
        return 'Night (19:00 - 07:00)'

periodDurations = {
    'Morning peak (07:00 - 10:00)': 3,   # 7 - 10
    'Inter-peak (10:00 - 16:00)': 6,     # 10 - 16
    'Evening peak (16:00 - 19:00)': 3,   # 16 - 19
    'Night (19:00 - 07:00)': 12
}

allPeriods = list(periodDurations.keys())

for sensorName in sensorsToSummarise.keys():
    dateIndex = []
    directionIndex = []
    summaryData = []
    
    dfSensor = peopleCountFrames[sensorName].copy()
    dfSensor.insert(0, 'Date', dfSensor.index.to_series().apply(lambda t: t.date()))
    dfSensor.insert(0, 'Day of week', dfSensor.index.to_series().apply(lambda t: t.strftime('%A')))
    dfSensor.insert(1, 'Time of day', dfSensor.index.to_series().apply(lambda t: t.strftime('%H:%M:%S')))
    dfSensor.insert(1, 'Period', dfSensor.index.to_series().apply(classifyTime))

    dfDailyPeriodTotals = dfSensor.groupby(['Date', 'Day of week', 'Period'], as_index=False).sum()
    dfAveragePeriodTotals = dfDailyPeriodTotals.groupby(['Day of week', 'Period'], as_index=False).median()
    
    for d in range(9, -1, -1):
        sensorDate = dateToday - pd.Timedelta(days=d)
        dateIndex.extend(np.repeat(sensorDate.strftime('%A %d %B'), 2))

        directionColumns = dfSensor.select_dtypes(plottableTypes).columns
        directionIndex.extend(directionColumns)
                         
        dfSensorOnDate = dfSensor.loc \
            [(sensorDate <= dfSensor.index) &
            (dfSensor.index < sensorDate + pd.Timedelta(hours=24))] \
            .copy() \
            .groupby(['Period']) \
            .agg(['sum', 'count']) \
            [directionColumns]
        dfSensorLastWeek = dfSensor.loc \
            [(sensorDate - pd.Timedelta(days=7) <= dfSensor.index) &
            (dfSensor.index < sensorDate - pd.Timedelta(days=7) + pd.Timedelta(hours=24))] \
            .copy() \
            .groupby(['Period']) \
            .agg(['sum', 'count']) \
            [directionColumns]
        dfSensorYesterday = dfSensor.loc \
            [(sensorDate - pd.Timedelta(days=1) <= dfSensor.index) &
            (dfSensor.index < sensorDate - pd.Timedelta(days=1) + pd.Timedelta(hours=24))] \
            .copy() \
            .groupby(['Period']) \
            .agg(['sum', 'count']) \
            [directionColumns]

        dfSensorAverageDayOfWeek = dfAveragePeriodTotals[dfAveragePeriodTotals['Day of week'] == sensorDate.strftime('%A')]

        for direction in directionColumns:
            summaryRow = []
            for period in allPeriods:
                periodStats = dfSensorOnDate[dfSensorOnDate.index == period][direction]
                if not periodStats['sum'].empty:

                    # Convert to an hourly value
                    periodTotal = periodStats['sum'].values[0]
                    periodHourly = periodTotal / periodStats['count'].values[0] * (3600 / peopleCountInterval)

                    # Change on yesterday
                    yesterdayHourly = dfSensorYesterday[dfSensorYesterday.index == period][direction]
                    yesterdayHourly = (yesterdayHourly['sum'].values[0] / yesterdayHourly['count'].values[0]) * (3600 / peopleCountInterval)
                    yesterdayChange = (periodHourly - yesterdayHourly) / yesterdayHourly

                    # Change on last week
                    lastWeekHourly = dfSensorLastWeek[dfSensorLastWeek.index == period][direction]
                    lastWeekHourly = (lastWeekHourly['sum'].values[0] / lastWeekHourly['count'].values[0]) * (3600 / peopleCountInterval)
                    lastWeekChange = (periodHourly - lastWeekHourly) / lastWeekHourly

                    # Change on normal profile
                    profileHourly = dfSensorAverageDayOfWeek[dfSensorAverageDayOfWeek['Period'] == period][direction]
                    profileHourly = profileHourly.values[0] / periodDurations[period]
                    profileChange = (periodHourly - profileHourly) / profileHourly

                    summaryRow.extend([
                        periodHourly, # Total
                        yesterdayChange, # Change on yesterday
                        lastWeekChange, # Change on last week
                        profileChange, # Change on average
                    ])
                else:
                    summaryRow.extend(np.repeat(0.0, 4))     
            summaryData.append(summaryRow)

    rowIndex = pd.MultiIndex.from_arrays([
            dateIndex,
            directionIndex
        ],
        names=['Date', 'Direction']
    )

    formattersSummary = {}
    colPeriods = []
    colStats = []
    for period in allPeriods:
        formattersSummary[(period, 'Hourly average flow')] = '{:,.0f}'
        formattersSummary[(period, 'Change from day before (%)')] = '{:+,.0%}'
        formattersSummary[(period, 'Change from week before (%)')] = '{:+,.0%}'
        formattersSummary[(period, 'Change from annual average (%)')] = '{:+,.0%}'
        colPeriods.extend(np.repeat(period, 4))
        colStats.extend([
            'Hourly average flow',
            'Change from day before (%)',
            'Change from week before (%)',
            'Change from annual average (%)'
        ])

    colIndex = pd.MultiIndex.from_arrays(
        [colPeriods, colStats],
        names=['Period', 'Statistic']
    )

    dfSummary = pd.DataFrame(summaryData, columns=colIndex, index=rowIndex)
    dfSummaryStyler = dfSummary.style \
        .format(formattersSummary) \
        .set_caption(sensorsToSummarise[sensorName]) \
        .set_table_styles(
            [dict(selector="th",props=[('text-align', 'center')]),
                dict(selector="tr:nth-child(2) th.col_heading",
                     props=[('vertical-align', 'bottom'),
                            ('writing-mode', 'vertical-rl'),
                            ]),
             dict(selector="caption",props=[('font-weight', 'bold'), ('font-size', '120%')])
            ]
        )

    periodBarColours = {
        'Morning peak (07:00 - 10:00)': '#FFA07A50',
        'Inter-peak (10:00 - 16:00)': '#EE1F5F50',
        'Evening peak (16:00 - 19:00)': '#FFA07A50',
        'Night (19:00 - 07:00)': '#A0FF7A50'
    }

    for period in allPeriods:
        dfSummaryStyler.background_gradient(
            subset=[(period, 'Change from annual average (%)')],
            vmin=-1.0,
            vmax=1.0,
            cmap='PiYG'
        )
        dfSummaryStyler.bar(subset=[(period, 'Hourly average flow')], color=periodBarColours[period], vmin=0)

    display(HTML(dfSummaryStyler._repr_html_()))